# Configuration Migration Example

This notebook demonstrates how to migrate from the old configuration system to the new ConfigManager.

## Old Way (Deprecated)
The old system used direct imports and manual configuration creation:

In [ ]:
# OLD WAY - Direct configuration import (deprecated)
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))

from src.config import ManufacturerConfig
from src.manufacturer import WidgetManufacturer

# Manual configuration
config = ManufacturerConfig(
    initial_assets=10_000_000,
    asset_turnover_ratio=1.0,
    operating_margin=0.08,
    tax_rate=0.25,
    retention_ratio=1.0
)

manufacturer = WidgetManufacturer(config)
print(f"Old way - Assets: ${manufacturer.assets:,.0f}")

## New Way (Recommended)
The new system uses ConfigManager with profiles, modules, and presets:

In [ ]:
# NEW WAY - Using ConfigManager (recommended)
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))

from src.config_manager import ConfigManager
from src.manufacturer import WidgetManufacturer

# Load configuration from profile
manager = ConfigManager()
config = manager.load_profile("default")

# Access manufacturer config
manufacturer = WidgetManufacturer(config.manufacturer)
print(f"New way - Assets: ${manufacturer.assets:,.0f}")
print(f"Profile: {config.profile.name}")
print(f"Description: {config.profile.description}")

## Benefits of the New System

### 1. Profile Variants
Easily switch between predefined configurations:

In [ ]:
# Load different profiles
profiles = ["default", "conservative", "aggressive"]

for profile_name in profiles:
    config = manager.load_profile(profile_name)
    print(f"\n{profile_name.upper()} Profile:")
    print(f"  Operating margin: {config.manufacturer.operating_margin:.1%}")
    print(f"  Tax rate: {config.manufacturer.tax_rate:.1%}")
    print(f"  Growth rate: {config.growth.annual_growth_rate:.1%}")

### 2. Runtime Overrides
Override specific parameters without changing files:

In [ ]:
# Override specific parameters
config = manager.load_profile(
    "default",
    manufacturer={"operating_margin": 0.12},
    growth={"annual_growth_rate": 0.15}
)

print("Configuration with overrides:")
print(f"  Operating margin: {config.manufacturer.operating_margin:.1%} (overridden)")
print(f"  Growth rate: {config.growth.annual_growth_rate:.1%} (overridden)")
print(f"  Tax rate: {config.manufacturer.tax_rate:.1%} (default)")

### 3. Presets for Common Scenarios
Apply predefined market conditions or risk scenarios:

In [ ]:
# Apply market condition presets
config_hard = manager.load_profile("default", presets=["hard_market"])
config_soft = manager.load_profile("default", presets=["soft_market"])

print("Market Condition Presets:")
print(f"  Hard market - Insurance available: {hasattr(config_hard, 'insurance')}")
print(f"  Soft market - Insurance available: {hasattr(config_soft, 'insurance')}")

# Apply multiple presets
config = manager.load_profile(
    "default",
    presets=["hard_market", "high_volatility"]
)
print("\nWith multiple presets applied")

### 4. Module Composition
Load only the modules you need:

In [ ]:
# Load specific modules only
config = manager.load_profile(
    "default",
    modules=["insurance", "stochastic"]
)

print("Loaded modules:")
print(f"  Insurance: {hasattr(config, 'insurance') and config.insurance is not None}")
print(f"  Stochastic: {hasattr(config, 'stochastic') and config.stochastic is not None}")
print(f"  Losses: {hasattr(config, 'losses') and config.losses is not None}")

## Migration Checklist

When updating your notebooks:

1. ✅ Replace `from src.config import ManufacturerConfig` with `from src.config_manager import ConfigManager`
2. ✅ Load configurations using `manager.load_profile()` instead of manual creation
3. ✅ Access sub-configurations through the loaded config (e.g., `config.manufacturer`)
4. ✅ Use profile names: `"default"` instead of `"baseline"`, `"aggressive"` instead of `"optimistic"`
5. ✅ Leverage presets and overrides instead of modifying YAML files

## Backward Compatibility

The old ConfigLoader still works but shows deprecation warnings:

In [ ]:
# This still works but is deprecated
from src.config_loader import ConfigLoader

loader = ConfigLoader()
config = loader.load("baseline")  # Shows deprecation warning

print("ConfigLoader still works for backward compatibility")
print(f"Manufacturer margin: {config.manufacturer.operating_margin:.1%}")

## Summary

The new ConfigManager provides:
- **Cleaner code**: Load complete configurations with one line
- **Flexibility**: Override parameters at runtime
- **Reusability**: Use profiles and presets across notebooks
- **Performance**: Built-in caching for faster loads
- **Maintainability**: Centralized configuration management

See `migration_guide.md` for complete documentation.